# Integer Manipulation - Algorithms
This prjoect looks at the mechanisms for different integer arithmatic by implementing different algorithms for integer manipulation. The initial focus is on the ['Karatsuba Multiplication Algorithm'](https://en.wikipedia.org/wiki/Karatsuba_algorithm). It turns out that multiplication using the algorithm we learned in school (shift across, multiply each digit, carry over, etc) is not the most efficient, especially when dealing with very large integers - such as those used in cryptography.

First, the algorithm is tested out in Python integers. Then, a ```BaseInteger``` class will be developed that will take in different representations and build up to addition, multiplication, different bases (from binary to base 36), powers, subtraction and division. Various operators, such as +, \*, - will be added. 

## Initial Algorithm
A faster algorithm can be implemented with a 'divide and conquer' approach. We split each of the numbers into two, so that:
\begin{align}
x \times  y &= (x_{1} \times  10^{n} + x_{0}) \times (y_{1} \times  10^{n} + y_{0}) \\
& = x_{1}y_{1} \times  10^{2n} + (x_{1}y_{0} + x_{0}y_{1}) \times 10^{n}) + x_{0}y_{0} \\
& = z_{2} \times  10^{2n} + z_{1} \times 10^{n} + z_{0} \\
\end{align}
where,
\begin{align}
z_{2} &= x_{1}y_{1} \\
z_{1} &= x_{1}y_{0} + x_{0}y_{1} \\
z_{0} &= x_{0}y_{0}
\end{align}

Karatsuba further recognised that these 4 multiplications could be reduced to 3 multiplications, because:
\begin{align}
z_{1} &= (x_{1} + x_{0})(y_{1} + y_{0}) - z_{2} - z_{0}
\end{align}

Note also that these are in base 10, but any base could be used.

### Splitting at 'n/2'
The first implementation of the algorithm using Python modular division. The ```split_at``` function will help to do the split the number at a given number of digits. e.g. 123987 split at 3 digits will give 123 and 987

In [9]:
def split_at(num,digits):
    return (num//digits, num % digits)

In [10]:
import math
print(split_at(123456,10**3))
print(split_at(123456,10**4))
print (math.log10(1_234_567_890)//2+1)
print(split_at(1_234_567_890,10**(math.ceil(math.log10(1_234_567_890))//2)))

(123, 456)
(12, 3456)
5.0
(12345, 67890)


### Initial version of Karatsuba
Now, here's a first implementation, using log10 from the Python math library to find the best split point for now. Later, this will be replaced, but for now it gives a quick way to find 'n/2', where n is the number of digits.

In [11]:
def RecursiveIntegerMultiply(x, y):
    # If one is under 10, this is the base case:
    if (x < 10) or (y < 10):
        return x * y

    # Find the number of (base 10) digits:
    m = int(math.log10(max(x,y))//2+1)
    split_mult = 10**m
    
    high_x,low_x = split_at(x,split_mult)
    high_y,low_y = split_at(y,split_mult)

    z2 = RecursiveIntegerMultiply(high_x,high_y)
    z1 = RecursiveIntegerMultiply(high_x+low_x, high_y+low_y)
    z0 = RecursiveIntegerMultiply(low_x,low_y)

    return z2*(split_mult**2)+(z1-z2-z0)*split_mult+z0

In [12]:
print(RecursiveIntegerMultiply(1000,1000))

1000000


In [13]:
for x,y in (123456789,987654321), \
           (123456789123456789123456789123456789,987654321987654321987654321), \
           (987654321987654321987654321123456789123456789123456789123, \
            987654321987654321987654321123456789123456789123456789123):
    print("Python:  ",x*y)
    print("Function:",RecursiveIntegerMultiply(x,y), end='\n\n')


Python:   121932631112635269
Function: 121932631112635269

Python:   121932631356500531591068431703703700581771069347203169112635269
Function: 121932631356500531591068431703703700581771069347203169112635269

Python:   975461059740893159506325257564700502268404202972107904421746680684042979980339279277425090639992990874560891109129
Function: 975461059740893159506325257564700502268404202972107904421746680684042979980339279277425090639992990874560891109129



## Implementation of BaseInteger class with add() for addition
This class will implement the ```add()``` method for addition, as well as initialising with a number as a string. Representing a number as a string means we can easily add different bases (e.g. hex) later. The ```__str__``` method ensures the number can be printed easily.

Internally, the design uses an integer array, with the digits in reverse order - so '125' is stored as \[5, 2, 1\]. This makes addition and carry over easier, as if we carry over to a higher digit, it can be added easily. The array used is a numpy array set to be int8. Although this implementation will not be 'blazing fast', at least this gives a fast underlying mechanism as numpy is fast and uses fixed areas in memory for the numbers. It also allows for easy implementation of the 'split_at' function and a 'shift' left/right function later.

In [28]:
import numpy as np
class BaseInteger():
    def __init__ (self, x):
        x = x.lstrip('0')
        if len(x) < 1: x = '0'
        self._number = np.zeros(len(x), dtype=np.int8)
        i = 0
        for digit in reversed(x):
            self._number[i] = int(digit)
            i += 1

    def __str__ (self):
        return ''.join(map(str,reversed(self._number.tolist())))
        
    def add (self, other):
        shorter,longer = self._number, other._number
        if len(shorter)>len(longer):
            longer,shorter = shorter,longer

        result = np.zeros(len(longer)+1,dtype=np.int8)
        carry = 0
        for digit in range(len(shorter)):
            add = shorter[digit]+longer[digit]+carry
            result[digit]=add%10
            carry = add//10
        for digit in range(len(shorter),len(longer)):
            add = longer[digit]+carry
            result[digit]=add%10
            carry = add//10
        if carry > 0:
            result[digit]=carry
            
        return BaseInteger(''.join(map(str,reversed(result))))


In [29]:
for test1, test2 in ('51231235234', '0'), ('51231235234', '51231235234'):
    int1 = BaseInteger(test1)
    int2 = BaseInteger(test2)
    res = int1.add(int2)
    print (int1, '+', int2, '=', res)

51231235234 + 0 = 51231235234
51231235234 + 51231235234 = 12462470468


## Implementing multiply() and adding support for different bases
The first implementation of the multiply. Note that this is not Karatsuba, as there isn't a subtract method to use! Still, the code checks for this and uses the longer, 4-multiplication mechanism otherwise.

Bases are implemented up to base 36. This is chosen, because the digits 0-9 plus letters A-Z allow 36 unique digits. Everything is changed to upper case. Note that multiplying/adding between bases works if the target bases get 'smaller', but the algorithm doesn't correctly deal with 'carryunder' yet - when the bases get larger and so need adjusting for higher digits to be represented correctly.

The constructor also adds support for string, numpy arrays and integers to be passed in.

In [30]:
import numpy as np

class BaseInteger():
    def __init__ (self, x, base=10):
        self._base = base
        if isinstance(x,(np.ndarray,)):
            self._number = x.copy()
        elif isinstance(x,(np.int,int)):
            self._number = np.asarray((x,),dtype=np.int16)
        else:
            if len(x) < 1:
                self._number = np.zeros(1, dtype=np.int16)
            elif (base <= 10): # string assumed!
                self._number = np.array(list(map(int,reversed(x))), dtype=np.int16)
            else: # could have characters in there.
                self._number = np.array(list(map(lambda x: (ord(x) - 48) if 48 <=ord(x)<=57 else ord(x)-65+10,reversed(x.upper()))), dtype=np.int16)

        # Calculate any carryovers
        self._number = self.propogate_carryovers(self._number,base)
        # Clean up any leading zeros:
        while len(self._number) > 1 and self._number[-1] == 0:
            self._number = np.delete(self._number, -1)
        # And a sanity check that all digits are less than the base number
        assert (self._number[self._number[:] >= base].sum() == 0), "Overflow - digit out of range."

    def __str__ (self):
        if (self._base <= 10):
            return ''.join(map(str,reversed(self._number.tolist())))
        else:
            return ''.join(map(lambda x: str(x) if x < 10 else chr(65+x-10), reversed(self._number.tolist())))
        
    def get_longer_shorter(self,x,y):
        if len(x)>len(y):
            return x,y
        return y,x

    def propogate_carryovers(self,result,base):
        # Calculate any carryovers
        for digit in range(0,len(result)-1):
            result[digit+1] += result[digit]//base
            result[digit] %= self._base
        while result[-1]>=self._base:
            result = np.append(result, [result[-1]//self._base])
            result[-2] %= self._base

        return result

    def shift (self, shift_num):
        result = BaseInteger(self._number,self._base)
        if shift_num == 0:
            return result
        elif shift_num > 0:
            result._number = np.append(np.zeros(shift_num, dtype=np.int16),result._number)
        elif len(result._number) + shift_num > 0:
            result._number = np.delete(result._number,range(0,-shift_num))
        else:
            return BaseInteger(0,self._base)
        return result

    def add (self, other):
        longer,shorter=self.get_longer_shorter(self._number,other._number)
        # Add the digits
        result = shorter[:] + longer [:len(shorter)]
        result = np.append(result, longer[len(shorter):])

        # Calculate any carryovers
        result = self.propogate_carryovers(result,self._base)
            
        return BaseInteger(result,self._base)


    def multiply(self, other):
        longer,shorter=self.get_longer_shorter(self._number,other._number)
        # If one is single digit, this is the base case:
        if len(shorter) == 1:
            return BaseInteger(longer[:] * shorter[0],self._base)

        # Split the digits into two halves:
        m = len(longer) >> 1 # divide by 2
        b = BaseInteger(longer[:m],self._base)
        a = BaseInteger(longer[m:],self._base)
        d = BaseInteger(shorter[:m],self._base)
        c = BaseInteger(shorter[m:],self._base)

        z2 = a.multiply(c)
        z0 = b.multiply(d)
        subtract = getattr(self, "subtract", None)
        if callable(subtract):
            # If the subtract function has been implemented,
            # use the version with 3 recursive calls and subtractions
            z1 = (a.add(b)).multiply(c.add(d))
            return z2.shift(2*m)+(z1-z2-z0).shift(m)+z0
        else:
            # Otherwise use the version with 4 recursive calls and additions
            z1a = a.multiply(d)
            z1 = b.multiply(c)
            return z2.shift(2*m).add((z1.add(z1a)).shift(m)).add(z0)


In [31]:
# Test cases
print ("Testing BaseInteger class - base 10")
for x,y in ('0','0'), ('1','1'), ('1','10'),('9','9'), ('1000','999'), \
           ('9999999','9999999'), ('9999999','1'), ('1', '9999999'), ('9999999','0'), \
           ('987654321987654321987654321987654321987654321','9'), \
           ('987654321987654321987654321987654321987654321',
            '987654321987654321987654321987654321987654321'):
    x_test = BaseInteger(x)
    y_test = BaseInteger(y)
    result = x_test.add(y_test)
    print ("Adding:",x,'+',y,'=',result, '- Result same as python:', \
           (int(x)+int(y) == int(result.__str__())) )

print ("\nTesting digit shift algorithm")
shifty = BaseInteger(2)
print('Shift 2 left 1:',shifty.shift(1))
print('Shift 2 left 5:',shifty.shift(5))
print('Shift 2 right 1:',shifty.shift(-1))
print('Shift 1000 right 3:',BaseInteger(1000).shift(-3))
print('Shift 2 left 5 and right 1:',shifty.shift(5).shift(-4))

print ("\nTesting Karatsuba multiplication algorithm")
for x,y in ('0','0'), ('1','1'), ('1','10'),('9','9'), ('1000','999'), \
           ('9999999','9999999'), ('9999999','1'), ('1', '9999999'), ('9999999','0'), \
           ('987654321987654321987654321987654321987654321','9'), \
           ('987654321987654321987654321987654321987654321',
            '987654321987654321987654321987654321987654321'):
    x_test = BaseInteger(x)
    y_test = BaseInteger(y)
    result = x_test.multiply(y_test)
    print ("Multiplying:",x,'x',y,'=',result, '- Result same as python:', \
           (int(x)*int(y) == int(result.__str__())) )
print ()
print('Testing alternative bases:')
print(BaseInteger('100',base=2))
print(BaseInteger('100',base=2).add(BaseInteger('100',base=2)))
print(BaseInteger('10000',base=2).add(BaseInteger('100101',base=2)))
print(BaseInteger('111',base=2).add(BaseInteger('111',base=2)),bin(7),bin(7+7))
print(BaseInteger('111',base=2).multiply(BaseInteger('111',base=2)),bin(7),bin(7*7))
print('  ',BaseInteger('11110100001000111111',base=2).multiply(BaseInteger('11110100001000111111',base=2)),bin(999999),"\n",bin(999999*999999))
print(BaseInteger('123434561',base=8).multiply(BaseInteger('234267123',base=8)))
print('  ',BaseInteger('A',base=16).multiply(BaseInteger('A',base=16)), hex(100))
print('  ',BaseInteger('BADF00D',base=16).multiply(BaseInteger('BADF00D',base=16)))
print('  ',BaseInteger('FFFF',base=16).add(BaseInteger('FFFF',base=16)))
print('  ',BaseInteger('FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF',base=16).add(BaseInteger('FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF',base=16)))
print('  ',BaseInteger('FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF',base=16).multiply(BaseInteger('FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF',base=16)))

print('  ',BaseInteger('ABCDEFGHIJKLMNOPQRSTUVWXYZ',base=26).add(BaseInteger('ABCDEFGHIJKLMNOPQRSTUVWXYZ',base=26)))
print('  ',BaseInteger('ABCDEFGHIJKLMNOPQRSTUVWXYZ',base=26).multiply(BaseInteger('ABCDEFGHIJKLMNOPQRSTUVWXYZ',base=26)))
print('  ',BaseInteger('ABCDEFGHIJKLMNOPQRSTUVWXYZ'[::-1],base=26).multiply(BaseInteger('ABCDEFGHIJKLMNOPQRSTUVWXYZ'[::-1],base=26)))

Testing BaseInteger class - base 10
Adding: 0 + 0 = 0 - Result same as python: True
Adding: 1 + 1 = 2 - Result same as python: True
Adding: 1 + 10 = 11 - Result same as python: True
Adding: 9 + 9 = 18 - Result same as python: True
Adding: 1000 + 999 = 1999 - Result same as python: True
Adding: 9999999 + 9999999 = 19999998 - Result same as python: True
Adding: 9999999 + 1 = 10000000 - Result same as python: True
Adding: 1 + 9999999 = 10000000 - Result same as python: True
Adding: 9999999 + 0 = 9999999 - Result same as python: True
Adding: 987654321987654321987654321987654321987654321 + 9 = 987654321987654321987654321987654321987654330 - Result same as python: True
Adding: 987654321987654321987654321987654321987654321 + 987654321987654321987654321987654321987654321 = 1975308643975308643975308643975308643975308642 - Result same as python: True

Testing digit shift algorithm
Shift 2 left 1: 20
Shift 2 left 5: 200000
Shift 2 right 1: 0
Shift 1000 right 3: 1
Shift 2 left 5 and right 1: 20

T

## Speed tests
The Python implementation is expected to be a lot faster than this one. Still, let's see:

In [32]:
%%timeit
975461059740893159506325259271757359037189456851699439086267339320835239555403139789971041*975461059740893159506325259271757359037189456851699439086267339320835239555403139789971041

199 ns ± 3.08 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [33]:
%%timeit
BaseInteger('975461059740893159506325259271757359037189456851699439086267339320835239555403139789971041').multiply \
(BaseInteger('975461059740893159506325259271757359037189456851699439086267339320835239555403139789971041'))

449 ms ± 16.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In fact, our implementation is about 2500 times slower. Still not bad to say that Python is using fast C implementation underneath.

## Adding infix operators
We now update our implementation to add infix operators. The functions to do this are listed in the [Python 3 documentation on emulating numeric types](https://docs.python.org/3/reference/datamodel.html#emulating-numeric-types)

In [34]:
# Create a new class that extends the BaseInteger class and adds in the operator support for + and *
class BaseInteger2(BaseInteger):
    def __init__ (self, x, base=10):
        super().__init__(x,base)

    def __add__(self, other):
        return self.add(other)
    def __mul__(self, other):
        return self.multiply(other)
        

In [35]:
# Test cases
print ("Testing + infix operator for BaseInteger2 class")
for x,y in ('0','0'), ('1','1'), ('1','10'),('9','9'), ('1000','999'), \
           ('9999999','9999999'), ('9999999','1'), ('1', '9999999'), ('9999999','0'), \
           ('987654321987654321987654321987654321987654321','9'), \
           ('987654321987654321987654321987654321987654321',
            '987654321987654321987654321987654321987654321'):
    x_test = BaseInteger2(x)
    y_test = BaseInteger2(y)
    result = x_test + y_test
    print ("Adding:",x,'+',y,'=',result, '- Result same as python:', \
           (int(x)+int(y) == int(result.__str__())) )

#print ("\nTesting digit shift algorithm")
#shifty = BaseInteger2(2)
# Implement __rshift__ for >> and __lshift__ for <<
#print('Shift 2 left 1:',shifty.shift(1))
#print('Shift 2 left 5:',shifty.shift(5))
#print('Shift 2 right 1:',shifty.shift(-1))
#print('Shift 1000 right 3:',BaseInteger(1000).shift(-3))
#print('Shift 2 left 5 and right 1:',shifty.shift(5).shift(-4))

print ("\nTesting * infix operator - uses Karatsuba multiplication algorithm")
for x,y in ('0','0'), ('1','1'), ('1','10'),('9','9'), ('1000','999'), \
           ('9999999','9999999'), ('9999999','1'), ('1', '9999999'), ('9999999','0'), \
           ('987654321987654321987654321987654321987654321','9'), \
           ('987654321987654321987654321987654321987654321',
            '987654321987654321987654321987654321987654321'):
    x_test = BaseInteger2(x)
    y_test = BaseInteger2(y)
    result = x_test * y_test
    print ("Multiplying:",x,'x',y,'=',result, '- Result same as python:', \
           (int(x)*int(y) == int(result.__str__())) )
print ()
print('Testing alternative bases:')
print(BaseInteger2('100',base=2))
print(BaseInteger2('100',base=2) + BaseInteger2('100',base=2))
print(BaseInteger2('10000',base=2) + BaseInteger2('100101',base=2))
print(BaseInteger2('111',base=2) + BaseInteger2('111',base=2),bin(7),bin(7+7))
print(BaseInteger2('111',base=2) * BaseInteger2('111',base=2),bin(7),bin(7*7))
print('  ',BaseInteger2('11110100001000111111',base=2) * BaseInteger2('11110100001000111111',base=2),bin(999999),"\n",bin(999999*999999))
print(BaseInteger2('123434561',base=8) * BaseInteger2('234267123',base=8))
print('  ',BaseInteger2('A',base=16) * BaseInteger2('A',base=16)), hex(100)
print('  ',BaseInteger2('BADF00D',base=16) * BaseInteger2('BADF00D',base=16))
print('  ',BaseInteger2('FFFF',base=16) + BaseInteger2('FFFF',base=16))
print('  ',BaseInteger2('FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF',base=16) + BaseInteger2('FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF',base=16))
print('  ',BaseInteger2('FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF',base=16) * BaseInteger2('FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF',base=16))

print('  ',BaseInteger2('ABCDEFGHIJKLMNOPQRSTUVWXYZ',base=26) + BaseInteger2('ABCDEFGHIJKLMNOPQRSTUVWXYZ',base=26))
print('  ',BaseInteger2('ABCDEFGHIJKLMNOPQRSTUVWXYZ',base=26) * BaseInteger2('ABCDEFGHIJKLMNOPQRSTUVWXYZ',base=26))
print('  ',BaseInteger2('ABCDEFGHIJKLMNOPQRSTUVWXYZ'[::-1],base=26) * BaseInteger2('ABCDEFGHIJKLMNOPQRSTUVWXYZ'[::-1],base=26))

Testing + infix operator for BaseInteger2 class
Adding: 0 + 0 = 0 - Result same as python: True
Adding: 1 + 1 = 2 - Result same as python: True
Adding: 1 + 10 = 11 - Result same as python: True
Adding: 9 + 9 = 18 - Result same as python: True
Adding: 1000 + 999 = 1999 - Result same as python: True
Adding: 9999999 + 9999999 = 19999998 - Result same as python: True
Adding: 9999999 + 1 = 10000000 - Result same as python: True
Adding: 1 + 9999999 = 10000000 - Result same as python: True
Adding: 9999999 + 0 = 9999999 - Result same as python: True
Adding: 987654321987654321987654321987654321987654321 + 9 = 987654321987654321987654321987654321987654330 - Result same as python: True
Adding: 987654321987654321987654321987654321987654321 + 987654321987654321987654321987654321987654321 = 1975308643975308643975308643975308643975308642 - Result same as python: True

Testing * infix operator - uses Karatsuba multiplication algorithm
Multiplying: 0 x 0 = 0 - Result same as python: True
Multiplying: 1

## Subtract() and negative numbers
Implementing subtract also means supporting negative numbers. Once subtract is implemented, retest the multiplication mechanism as it will use the Karatsuba version when it detects a subtract function. 

## Powers

## Division
A simple implementation would be to repeatedly subtract the number! We'll first try the long division implementation from school. 

## Final testing and abstraction
